In [22]:
%pip install -q -r requirements.txt
%pip install -q "pymongo[srv]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
from imports import *
from functions import *
from IPython.core.display import *
from sqlalchemy import String
from urllib.parse import unquote

Initialize PostgreSQL Connection

In [24]:
path_postgresql_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\postgresql_creds.json"

with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "MyProjects"
schema = "End_To_End_Oceanography_ML"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
conn = engine.connect()

Get Data from APIs

Get Available Stations ID List

In [25]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


Filter Dysfunctional Stations

In [26]:
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift', "lost", "No sensor"]
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')
# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    
    # Vérifier si un des éléments de blacklist est dans n'importe quelle cellule de la ligne
    if any(blacklist_word.lower() in str(cell).lower() for cell in row for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
    else:
        # Si la station passe le test, afficher et ajouter le station_id à l'ensemble
        stations_id_set.add(station_id)

# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Après Filtre: 16


Build Dictionary of Stations

In [27]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']  # Récupérer l'ID de la bouée à partir de la colonne 'Station'
    buoy_datas[buoy_id] = {}
# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}")


Nombre de bouées traitées : 16


Columns Check Tests

In [28]:
df_marine_test = NDBC.realtime_observations(51002)
print(f'{df_marine_test.shape[0]} rows \n{df_marine_test.isna().sum()}')
df_marine_test.head(1)

6509 rows 
wind_direction               14
wind_speed                   12
wind_gust                    12
wave_height                2745
dominant_wave_period       4348
average_wave_period        2749
dominant_wave_direction    2749
pressure                     12
air_temperature              31
water_temperature          1368
dewpoint                     31
visibility                 6509
3hr_pressure_tendency      5415
water_level_above_mean     6509
time                          0
dtype: int64


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,time
0,90.0,8.0,9.0,1.9,9.0,5.7,90.0,1017.6,24.4,25.3,21.6,NaN,NaN,NaN,2025-03-19 16:20:00+00:00


In [29]:
df_meteo_test = meteo_api_request(coordinates=[12, 23])
print(f'{df_meteo_test.shape[0]} rows \n{df_meteo_test.isna().sum()}')
df_meteo_test.head(1)

2376 rows 
date                        0
temperature_2m            186
relative_humidity_2m      186
dew_point_2m              186
precipitation             186
rain                      186
showers                   186
pressure_msl              186
surface_pressure          186
cloud_cover               186
cloud_cover_low           186
cloud_cover_mid           186
cloud_cover_high          186
visibility                186
wind_speed_10m            186
soil_temperature_0cm      442
soil_moisture_0_to_1cm    442
is_day                      0
dtype: int64


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-12-17 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [30]:
metadata_extracted = get_station_metadata(51002)
metadata_extracted

{'Watch circle radius': '5029 yards',
 'Water depth': '4979 m',
 'Sea temp depth': '1.5 m below water line',
 'Barometer elevation': '2.7 m above mean sea level',
 'Anemometer height': '4.1 m above site elevation',
 'Air temp height': '3.7 m above site elevation',
 'Site elevation': 'sea level',
 'Location': '17.070 N 157.755 W (17°4\'12" N 157°45\'18" W)',
 'Statation Type': 'Owned and maintained by National Data Buoy Center, 3-meter foam buoy, SCOOP payload',
 'Name': 'Station 51002 (LLNR 28005.1) - SOUTHWEST HAWAII - 215NM SSW of Hilo, HI'}

In [31]:
station_name, station_id, station_zone, lat_buoy, lon_buoy, marine_data_table_name = parse_buoy_json(metadata_extracted)
print(f'{station_name} {station_id} {station_zone} {lat_buoy}{lon_buoy}\n {marine_data_table_name}')

station 51002 (llnr 28005.1) 51002 southwest hawaii 17.07-157.755
 station_51002_southwest_hawaii_17-07_-157-755


Big API Call Loop

In [32]:
list_bronze_table_pairs = []

for buoy_id in buoy_datas:  # Boucle sur les bouées dans le dictionnaire
    # Récupérer les métadonnées de la bouée
    metadata_extracted = get_station_metadata(buoy_id)
    
    station_name, station_id, station_zone, lat, lon, table_name = parse_buoy_json(metadata_extracted)

    
    buoy_datas[buoy_id]["Station Name"] = station_name
    buoy_datas[buoy_id]["Station ID"] = station_id
    buoy_datas[buoy_id]["Zone"] = station_zone
    buoy_datas[buoy_id]["Lat"] = lat
    buoy_datas[buoy_id]["Lon"] = lon
    Bronze_Marine_Table_Name =buoy_datas[buoy_id]["Bronze Marine Table Name"] = "bronze_marine_data_" + table_name
    Bronze_Meteo_Table_Name =buoy_datas[buoy_id]["Bronze Meteo Table Name"] = "bronze_meteo_data_" + table_name
    duo = [Bronze_Marine_Table_Name, Bronze_Meteo_Table_Name]

    list_bronze_table_pairs.append(duo)

    # NOAA API CALL
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            print(f"⚠️ Marine Data is empty for buoy {buoy_id}")

        buoy_datas[buoy_id]["Marine Dataframe"] = df_marine
        print(f"🌊 Marine Data Successfully collected for buoy {buoy_id}")

        # Loading Marine Data into PostgreSQL Tables
        try :
            rows_before = df_marine.shape[0]
            print(f"🔄 Bouée {buoy_id} : Insertion des données.")
            # Charger les données dans la table
            load_data_in_table(conn=conn, df=df_marine, schema=schema, table_name=Bronze_Marine_Table_Name, key_column="time")
            query = text(f'SELECT COUNT(*) FROM "{schema}"."{Bronze_Marine_Table_Name}"')
            result = conn.execute(query)
            rows_after = result.scalar()
            print(f"✅ {buoy_id} : La table '{Bronze_Marine_Table_Name}' contient désormais {rows_after} lignes (df : {rows_before} lignes).")
        except Exception as e:
            print(f"⚠️ Bouée {buoy_id} : Ereur à l'insertion des données:\n{e}\n")

    except Exception as e:
        print(f"❌ Failed to collect Marine Data for buoy {buoy_id}: \n{e}\n")


    # Open-Meteo API Call 
    try:
        df_meteo = meteo_api_request(coordinates=[lat, lon])
        if df_meteo is None or df_meteo.empty:
            print(f"⚠️ Meteo Data is empty for buoy {buoy_id}")
            continue

        buoy_datas[buoy_id]["Meteo Dataframe"] = df_meteo
        print(f"🌦️ Meteo Data Successfully collected for buoy {buoy_id}")


        # Loading Meteo Data into PostgreSQL Tables
        try :
            rows_before = df_meteo.shape[0]
            print(f"🔄 Bouée {buoy_id} : Insertion des données.")
            # Charger les données dans la table
            load_data_in_table(conn=conn, df=df_meteo, schema=schema, table_name=Bronze_Meteo_Table_Name, key_column="date")
            query = text(f'SELECT COUNT(*) FROM "{schema}"."{Bronze_Meteo_Table_Name}"')
            result = conn.execute(query)
            rows_after = result.scalar()
            print(f"✅ {buoy_id} : La table '{Bronze_Meteo_Table_Name}' contient désormais {rows_after} lignes (df : {rows_before} lignes).")
        except Exception as e:
            print(f"⚠️ Bouée {buoy_id} : Ereur à l'insertion des données:\n{e}\n")


    except Exception as e:
        print(f"❌ Failed to collect Meteo Data for buoy {buoy_id}: \n{e}\n")
        continue

buoy_datas["Bronze Tables List"] = list_bronze_table_pairs

🌊 Marine Data Successfully collected for buoy 44020
🔄 Bouée 44020 : Insertion des données.

Schema "End_To_End_Oceanography_ML" already exists.
Table 'bronze_marine_data_station_44020_nantucket_sound_41-497_-70-283' already exists.
Table is not empty, avoiding duplicates...
No new data to insert.
✅ 44020 : La table 'bronze_marine_data_station_44020_nantucket_sound_41-497_-70-283' contient désormais 6518 lignes (df : 6518 lignes).
🌦️ Meteo Data Successfully collected for buoy 44020
🔄 Bouée 44020 : Insertion des données.

Schema "End_To_End_Oceanography_ML" already exists.
Table 'bronze_meteo_data_station_44020_nantucket_sound_41-497_-70-283' already exists.
Table is not empty, avoiding duplicates...
No new data to insert.
✅ 44020 : La table 'bronze_meteo_data_station_44020_nantucket_sound_41-497_-70-283' contient désormais 2376 lignes (df : 2376 lignes).
🌊 Marine Data Successfully collected for buoy 44025
🔄 Bouée 44025 : Insertion des données.

Schema "End_To_End_Oceanography_ML" alread

In [33]:
buoy_datas["Bronze Tables List"]

[['bronze_marine_data_station_44020_nantucket_sound_41-497_-70-283',
  'bronze_meteo_data_station_44020_nantucket_sound_41-497_-70-283'],
 ['bronze_marine_data_station_44025_long_island_40-258_-73-175',
  'bronze_meteo_data_station_44025_long_island_40-258_-73-175'],
 ['bronze_marine_data_station_44065_new_york_harbor_entrance_40-368_-73-701',
  'bronze_meteo_data_station_44065_new_york_harbor_entrance_40-368_-73-701'],
 ['bronze_marine_data_station_46006_southeast_papa_40-764_-137-377',
  'bronze_meteo_data_station_46006_southeast_papa_40-764_-137-377'],
 ['bronze_marine_data_station_46014_pt_arena_39-225_-123-98',
  'bronze_meteo_data_station_46014_pt_arena_39-225_-123-98'],
 ['bronze_marine_data_station_46025_santa_monica_basin_33-755_-119-045',
  'bronze_meteo_data_station_46025_santa_monica_basin_33-755_-119-045'],
 ['bronze_marine_data_station_46027_st_georges_41-84_-124-382',
  'bronze_meteo_data_station_46027_st_georges_41-84_-124-382'],
 ['bronze_marine_data_station_46053_east

In [34]:
def explore_dict(d, indent=0):
    """ Fonction récursive pour afficher toute la structure du dictionnaire """
    for key, value in d.items():
        print(" " * indent + f"- {key}: {type(value)}")
        if isinstance(value, dict):
            explore_dict(value, indent + 4)  # Explorer récursivement avec une indentation
        elif isinstance(value, list):
            if len(value) > 0:
                print(" " * (indent + 4) + f"Liste ({len(value)} éléments), type du premier élément: {type(value[0])}")
                if isinstance(value[0], dict):
                    explore_dict(value[0], indent + 8)  # Explorer si c'est une liste de dicts
        else:
            print(" " * (indent + 4) + f"Valeur: {value}")

# Récupérer la première clé du dictionnaire
first_key = next(iter(buoy_datas))
print(f"Exploration de la première clé: {first_key}\n")

# Exécuter la fonction sur le premier élément uniquement
explore_dict({first_key: buoy_datas[first_key]})


Exploration de la première clé: 44020

- 44020: <class 'dict'>
    - Station Name: <class 'str'>
        Valeur: station 44020 (llnr 13700)
    - Station ID: <class 'str'>
        Valeur: 44020
    - Zone: <class 'str'>
        Valeur: nantucket sound
    - Lat: <class 'float'>
        Valeur: 41.497
    - Lon: <class 'float'>
        Valeur: -70.283
    - Bronze Marine Table Name: <class 'str'>
        Valeur: bronze_marine_data_station_44020_nantucket_sound_41-497_-70-283
    - Bronze Meteo Table Name: <class 'str'>
        Valeur: bronze_meteo_data_station_44020_nantucket_sound_41-497_-70-283
    - Marine Dataframe: <class 'pandas.core.frame.DataFrame'>
        Valeur:       wind_direction  wind_speed  wind_gust  wave_height  \
0               10.0         9.0       10.0          NaN   
1               10.0         9.0       10.0          0.5   
2               10.0         9.0       10.0          0.5   
3              360.0         9.0       11.0          NaN   
4              360.

In [35]:
# Filtrer les entrées qui n'ont pas un 'buoy_id' valide (par exemple, 'Bronze Tables List')
buoy_datas_filtered = {buoy_id: data for buoy_id, data in buoy_datas.items() if buoy_id != "Bronze Tables List"}

# Maintenant, créer le dictionnaire 'buoy_metadata' à partir des données filtrées
buoy_metadata = {}

for buoy_id, data in buoy_datas_filtered.items():
    buoy_metadata[buoy_id] = {"Station ID": buoy_id}  # Ajouter l'ID comme colonne
    if isinstance(data, dict):  # Vérifier que c'est bien un dictionnaire
        for k, v in data.items():
            if isinstance(v, pd.DataFrame):
                buoy_metadata[buoy_id][k] = v.to_json()  # Convertir en JSON
            elif isinstance(v, list):
                buoy_metadata[buoy_id][k] = json.dumps(v)  # Convertir la liste en JSON
            else:
                buoy_metadata[buoy_id][k] = v  # Garder tel quel

# Créer le DataFrame
df_buoys_datas = pd.DataFrame.from_dict(buoy_metadata, orient='index').reset_index(drop=True)

# Afficher le résultat
df_buoys_datas

,Station ID,Station Name,Zone,Lat,Lon,Bronze Marine Table Name,Bronze Meteo Table Name,Marine Dataframe,Meteo Dataframe
0,44020,station 44020 (llnr 13700),nantucket sound,41.497,-70.283,bronze_marine_data_station_44020_nantucket_sou...,bronze_meteo_data_station_44020_nantucket_soun...,"{""wind_direction"":{""0"":10.0,""1"":10.0,""2"":10.0,...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
1,44025,station 44025 (llnr 830),long island,40.258,-73.175,bronze_marine_data_station_44025_long_island_4...,bronze_meteo_data_station_44025_long_island_40...,"{""wind_direction"":{""0"":30.0,""1"":40.0,""2"":40.0,...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
2,44065,station 44065 (llnr 725),new york harbor entrance,40.368,-73.701,bronze_marine_data_station_44065_new_york_harb...,bronze_meteo_data_station_44065_new_york_harbo...,"{""wind_direction"":{""0"":50.0,""1"":40.0,""2"":40.0,...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
3,46006,station 46006 (llnr 510),southeast papa,40.764,-137.377,bronze_marine_data_station_46006_southeast_pap...,bronze_meteo_data_station_46006_southeast_papa...,"{""wind_direction"":{""0"":290.0,""1"":290.0,""2"":300...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
4,46014,station 46014 (llnr 445),pt arena,39.225,-123.980,bronze_marine_data_station_46014_pt_arena_39-2...,bronze_meteo_data_station_46014_pt_arena_39-22...,"{""wind_direction"":{""0"":140.0,""1"":140.0,""2"":150...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
5,46025,station 46025 (llnr 181),santa monica basin,33.755,-119.045,bronze_marine_data_station_46025_santa_monica_...,bronze_meteo_data_station_46025_santa_monica_b...,"{""wind_direction"":{""0"":null,""1"":null,""2"":260.0...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
6,46027,station 46027 (llnr 562),st georges,41.840,-124.382,bronze_marine_data_station_46027_st_georges_41...,bronze_meteo_data_station_46027_st_georges_41-...,"{""wind_direction"":{""0"":160.0,""1"":160.0,""2"":150...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
7,46053,station 46053 (llnr 196),east santa barbara,34.241,-119.839,bronze_marine_data_station_46053_east_santa_ba...,bronze_meteo_data_station_46053_east_santa_bar...,"{""wind_direction"":{""0"":300.0,""1"":300.0,""2"":290...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
8,46072,station 46072 (llnr 27510),central aleutians 230 nm sw dutch harbor,51.645,-172.145,bronze_marine_data_station_46072_central_aleut...,bronze_meteo_data_station_46072_central_aleuti...,"{""wind_direction"":{""0"":260.0,""1"":260.0,""2"":260...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."
9,46078,station 46078 (llnr 1187),albatross bank,55.561,-152.599,bronze_marine_data_station_46078_albatross_ban...,bronze_meteo_data_station_46078_albatross_bank...,"{""wind_direction"":{""0"":140.0,""1"":130.0,""2"":130...","{""date"":{""0"":1734393600000,""1"":1734397200000,""..."


In [36]:
load_data_in_table(conn=conn, df= df_buoys_datas, schema=schema, table_name="buoys_datas", key_column= "Station ID")


Schema "End_To_End_Oceanography_ML" already exists.
Table 'buoys_datas' already exists.
Table is not empty, avoiding duplicates...
No new data to insert.


In [37]:
    # if df_meteo.empty or df_marine.empty:
    #     print(f"⚠️ Skipping merge for buoy {buoy_id}: One of the DataFrames is empty after processing")
    #     continue

    # else :
    #     # Resampling
    #     df_meteo = process_and_resample(df_meteo, column_name="date")
    #     df_marine = process_and_resample(df_marine, column_name="time")

    # # Fusion des DataFrames
    # try:
    #     df_merged = pd.merge(df_marine, df_meteo, on="Datetime", how="inner")
    #     if df_merged is None or df_merged.empty:
    #         print(f"⚠️ Merge result failed for buoy {buoy_id}.")

    #     elif df_merged.empty:
    #         print(f"⚠️ Merge result is empty for buoy {buoy_id}, possible mismatch in timestamps.")
    #     else:
    #         buoy_datas[buoy_id]["Merged Dataframe"] = df_merged
    #         print(f"🌊🌦️ Marine and Meteo Data Successfully merged together for buoy {buoy_id}\n")


    # except Exception as e:
    #     print(f"❌ Merge Failed for Buoy {buoy_id}: {e}")

In [38]:
# for key, value in buoy_datas.items():
#     if "Merged Dataframe" in value and "Bronze Table Name" in value:
#         df = value["Merged Dataframe"]
#         bronze_table_name = value["Bronze Table Name"]

#         # Afficher le nombre de lignes de la DataFrame avant l'insertion
#         rows_before = df.shape[0]
#         print(f"🔄 Bouée {key} : Insertion des données.")

#         # Charger les données dans la table
#         load_data_in_table(conn=conn, df=df, schema=schema, table_name=bronze_table_name, key_column="Datetime")

#         # Vérifier le nombre de lignes après insertion
#         query = text(f'SELECT COUNT(*) FROM "{schema}"."{bronze_table_name}"')
#         result = conn.execute(query)
#         rows_after = result.scalar()  # Obtient le nombre de lignes

#         # Afficher le nombre de lignes après insertion
#         print(f"✅ {key} : La table '{bronze_table_name}' contient désormais {rows_after} lignes (df : {rows_before} lignes).")
#     else:
#         print(f"⚠️ Bouée {key} : données fusionnées absentes, skipping load")

In [39]:
# openmongo_creds = r'C:\Users\f.gionnane\Documents\Data Engineering\Credentials\mongo_creds.json'

# with open(openmongo_creds, 'r') as file:
#     content = json.load(file)
#     mongo_user = content["user"]
#     mongo_password = content["password"]
#     mongo_string = content["connection_string"]

# uri = mongo_string
# # Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'))

# # Send a ping to confirm a successful connection
# try:
#     client.admin.command('ping')
#     print("Pinged your deployment. You successfully connected to MongoDB!")
# except Exception as e:
#     print(e)
#     print(f'{mongo_user}\n{mongo_password}\n{mongo_string}')

In [40]:
# import nest_asyncio
# nest_asyncio.apply()
# import asyncio
# import json
# from motor.motor_asyncio import AsyncIOMotorClient
# from pymongo.server_api import ServerApi
# import nest_asyncio  # ✅ Ajout de nest_asyncio

# # Appliquer le patch pour éviter l'erreur
# nest_asyncio.apply()

# openmongo_creds = r'C:\Users\f.gionnane\Documents\Data Engineering\Credentials\mongo_creds.json'

# with open(openmongo_creds, 'r') as file:
#     content = json.load(file)
#     mongo_user = content["user"]
#     mongo_password = content["password"]
#     mongo_string = content["connection_string"]

# async def ping_server():
#     uri = mongo_string
#     client = AsyncIOMotorClient(uri, server_api=ServerApi('1'))
    
#     try:
#         await client.admin.command('ping')
#         print("Pinged your deployment. You successfully connected to MongoDB!")
#     except Exception as e:
#         print(e)
#         print(f'{mongo_user}\n{mongo_password}\n{mongo_string}')

# # Lancer la fonction asynchrone sans asyncio.run()
# asyncio.get_event_loop().run_until_complete(ping_server())
